In [1]:
import pandas as pd
import dask.dataframe as dd
from glob import glob

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

/opt/miniconda3/envs/gtu_eval_parallel/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49657 instead
  warnings.warn(
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-wd2u3gp0', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-g3lj6mkp', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-ki02au95', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-8u7d2avu', purging


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:49657/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:49657/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:49658,Workers: 4
Dashboard: http://127.0.0.1:49657/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:49669,Total threads: 2
Dashboard: http://127.0.0.1:49670/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:49660,


In [3]:
zipcode_region = 9

In [4]:
df = dd.read_csv("./data/*.csv").compute()

In [5]:
df_shape = pd.read_csv(f"./zipcodes/zipcode_shape_{zipcode_region}.csv")

In [6]:
df = df.join(df_shape.set_index("gid"), on="gid")
df = df.dropna()
df = df.set_index("gid")
df.to_parquet(f"joined_{zipcode_region}")

# {
#   "type": "Feature",
#   "geometry": {
#     "type": "Point",
#     "coordinates": [125.6, 10.1]
#   },
#   "properties": {
#     "name": "Dinagat Islands"
#   }
# }

In [7]:
df

,zcta5ce10,avg,score_1,score_2,score_3,score_4,score_5,st_asgeojson
gid,,,,,,,,
100,95830,0.870382,19,51,57,182,181,"{""type"":""MultiPolygon"",""coordinates"":[[[[-121...."
10128,91766,0.821511,4100,8377,8493,16597,12449,"{""type"":""MultiPolygon"",""coordinates"":[[[[-117...."
10131,91767,0.811966,3407,8034,6934,13040,8917,"{""type"":""MultiPolygon"",""coordinates"":[[[[-117...."
10133,91768,0.823718,1824,4680,4612,10073,5854,"{""type"":""MultiPolygon"",""coordinates"":[[[[-117...."
10136,91770,0.811848,2752,7741,7386,13697,7007,"{""type"":""MultiPolygon"",""coordinates"":[[[[-118...."
...,...,...,...,...,...,...,...,...
9842,92220,0.826596,2138,3671,3300,6540,7400,"{""type"":""MultiPolygon"",""coordinates"":[[[[-116...."
9843,92222,0.860572,1,3,4,10,9,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114...."
9846,92223,0.787460,3448,5771,4750,7989,5346,"{""type"":""MultiPolygon"",""coordinates"":[[[[-117...."


In [8]:
def insert_data_into_json(row):
    new_geojson = {}
    new_geojson["type"] = "Feature"
    new_geojson["geometry"] = eval(row["st_asgeojson"])
    
    tem = {}
    tem['zipcode'] = row["zcta5ce10"]
    for score_val in range(1,6):
        tem[f"score_{score_val}"] = row[f"score_{score_val}"]

    tem["percentage_1"] = ((row[f"score_1"] )/(row[f"score_1"]+row[f"score_2"]+row[f"score_3"]+row[f"score_4"]+row[f"score_5"]))*100
    new_geojson["properties"] = tem
    
    row["st_asgeojson"] = new_geojson
    
    return row

In [9]:
df = pd.read_parquet(f"joined_{zipcode_region}")
df = df.apply(insert_data_into_json,axis=1)
df.to_parquet(f"converted_{zipcode_region}")

In [10]:
df = pd.read_parquet(f"converted_{zipcode_region}")
df['st_asgeojson'].to_json(f"zipcode_region_{zipcode_region}.json",orient="values")

In [11]:
prefix = '{"type": "FeatureCollection", "features": '
suffix = '}'

In [12]:
with open(f"zipcode_region_{zipcode_region}.json",'r') as f:
    read_data = f.read()
with open(f"zipcode_region_{zipcode_region}.json",'w') as f:
    write_data = prefix+read_data+suffix
    f.write(write_data)